# Importando Bibliotecas

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Bibliotecas necessárias
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import spacy
from spacy.lang.pt import stop_words
import random
import json
import os
import re
from itertools import chain
import sys
from __future__ import unicode_literals, print_function
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import base64
import csv

csv.field_size_limit(sys.maxsize)

!python -m spacy download pt_core_news_sm

2023-09-19 11:25:00.738846: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 40.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import *
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
!pip install unidecode
from unidecode import unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 5.5 MB/s eta 0:00:00


In [5]:
!pip install gensim
from gensim.models import Word2Vec

In [6]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

# Pré-processamento

In [7]:
# Ajusta a formatação dos textos
def remove_formatting(text):
    text = text.replace("\n", " ").replace("\t", " ")
    while text.find("  ") != -1:
        text = text.replace("  ", " ")
    return text

In [8]:
NLP = spacy.load("pt_core_news_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer", "ner"])
NLP.enable_pipe("senter")

def get_sentences(text: str):
    sentences = [sent.text.strip() for sent in NLP(text).sents]
    sentences = list(filter(lambda s: len(s) > 0, sentences))
    return sentences

In [9]:
# Remove acentos e passa para lowercase
formatar = lambda palavra: unidecode(str(palavra).lower())

STEMMER = SnowballStemmer("portuguese", ignore_stopwords=True)
STOPWORDS = stopwords.words('portuguese')

def extract_stem_tokens(sentence: str):
    tokens = [formatar(t) for t in word_tokenize(sentence)]
    important_tokens = list(filter(lambda t: not t in STOPWORDS and t.isalpha(), tokens))
    stems = [STEMMER.stem(t) for t in important_tokens]
    return stems

In [10]:
# Word2Vec
print("Carregando modelo Word2Vec...")
WORD2VEC = Word2Vec.load("/content/drive/Shareddrives/IA 2023 - Projeto 1 Grupo 3/Modelos/word2vec.model")

# TF-IDF
tf_idf_weights = {}
with open("/content/drive/Shareddrives/IA 2023 - Projeto 1 Grupo 3/Modelos/tf_idf_weights.csv", 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in tqdm(csv_reader, desc="Carregando tokens TF-IDF", position=0, leave=True):
        tf_idf_weights[row[0]] = float(row[1])

Carregando modelo Word2Vec...


Carregando tokens TF-IDF: 40727it [00:00, 68204.03it/s]


In [11]:
def vectorize_sentence(sentence: str):
    return vectorize_tokens(extract_stem_tokens(sentence))

def vectorize_tokens(tokens: list):
    if not tokens:
        return np.zeros_like(WORD2VEC.wv[0])
    tokens = list(filter(lambda t: (t in WORD2VEC.wv) and (t in tf_idf_weights.keys()), tokens))
    #tfidf = np.sqrt([[tf_idf_weights[t]] for t in tokens])
    tfidf = np.array([[tf_idf_weights[t]] for t in tokens])
    tfidf = np.log(tfidf / np.min(tfidf)) + 1
    w2v = np.array([WORD2VEC.wv[t] for t in tokens])
    return np.sum((w2v * tfidf) / sum(tfidf), axis=0) if len(tokens) > 0 else np.zeros_like(WORD2VEC.wv[0])

# Funcionamento

In [12]:
path_root              = "/content/drive/Shareddrives/IA 2023 - Projeto 1 Grupo 3/Modelos/"
publicacoes_model      = keras.models.load_model(f"{path_root}publicacoes_e56.keras")
publicacoes_head_model = keras.models.load_model(f"{path_root}publicacoes_head_e83.keras")
modalidades_model      = keras.models.load_model(f"{path_root}modalidades_e31.keras")
modalidades_head_model = keras.models.load_model(f"{path_root}modalidades_head_e10.keras")
festividade_head_model = keras.models.load_model(f"{path_root}festividade_head_e10.keras")

In [13]:
lista_modalidades = [
    'Pregão Presencial - Ata de Registro de Preços', 'Ata de Registro de Preço', 'Seleção Pública Simplificada', 'Aviso de Chamamento Público',
    'Dispensa de Licitação', 'Tomada de Preço', 'Ata de Registro de Preços', 'Chamada Pública', 'Pregão', 'Convite', 'Concorrência',
    'Credenciamento', 'Inexigibilidade', 'Pregão Presencial', 'Regime Diferenciado de Contratação', 'Pregão Eletrônico'
]
lista_tipo_publicacao = [
    'Convocação de Licitante', 'Resultado de Julgamento', 'Termo de Comodato', 'Emenda Parlamentar', 'Nomeação Conselho', 'Anulação de ato',
    'Publicação de resultado', 'Processo Administrativo', 'Extrato de ata de registro de preços', 'Requerimento', 'Instrução normativa',
    'Lei Aldir Blanc', 'Proposta de Lei', 'Sessão Solene', 'Extrato de Termos de Compromisso', 'Cooperação Técnica',
    'Autorização de movimentação bancária', 'Ata da Sessão Pública', 'Regimento', 'Convênio', 'Memorial Descritivo', 'Anexos', 'Resolução',
    'Aviso de Demolição - Suspensão', 'Aprovação da Programação Anual de Saúde', 'Cessão de Uso', 'Autorização de uso de imóvel', 'Notificação',
    'Extrato de ata', 'Leilão', 'Manifestação de interesse social', 'Licenciamento Ambiental', 'Rescisão Contratual', 'Programação financeira',
    'Homologação', 'Portaria', 'Convocação para participar de Conselho Municipal', 'Inexigibilidade', 'Aviso de Demolição', 'Termo Aditivo',
    'Ata de abertura de envelope', 'Processo de impeachment', 'Aviso de Licitação', 'Concessão de Patrocínio', 'Processo Administrativo Sanitário',
    'Nota Informativa', 'Processo Seletivo', 'Licitação Deserta', 'Errata', 'Extrato de Contrato', 'Lançamento de tributos', 'Edital de Proclamas',
    'Termo de ajuste de contas', 'Promoção de religião', 'Atestado publicação RGF', 'Desapropriação', 'Revogação', 'Decreto', 'Termo de Fomento',
    'Atos de Pessoal', 'Autorização de Aplicação e Resgate', 'Credenciamento', 'Sanção contratual', 'Carta de Advertência',
    'Reequilíbrio Econômico Financeiro', 'Termo de Colaboração', 'Certidão', 'Execução de Pagamento', 'Cancelamento ou adiantamento indeterminado',
    'Conferência Municipal', 'Parcelamento e Confissão de Débitos Previdenciários', 'Aviso de Advertência', 'Adesão a ata de registro de preços',
    'Concurso Público', 'Parecer em credenciamento', 'Ratificação de dispensa', 'Campeonato esportivo', 'Concurso cultural', 'Audiência Pública',
    'Eleições de membros', 'Julgamento de Contas', 'Ata de Reunião', 'Contencioso Administrativo Fiscal', 'Lei', 'Precatórios FUNDEF',
    'Promulgação de Lei', 'Protocolo de Intenções', 'Cronograma de datas para tramitação de Lei', 'Certidão de Regularização Fundiária',
    'Prorrogação de credenciamento', 'Notificação de recebimento de recursos'
]

In [14]:
def avaliar(publicacao: str, topn=3):
    with HiddenPrints():
        # Calcula os vetores de cada frase da publicação
        frases = get_sentences(remove_formatting(publicacao))
        vectors = np.array([vectorize_sentence(f) for f in frases])
        # Extrai a modalidade
        prob_modalidades = np.sum(modalidades_head_model.predict(vectors) * modalidades_model.predict(vectors), axis=0)
        top_modalidades = np.argsort(-prob_modalidades)[:topn]
        modalidades = [(lista_modalidades[i], prob_modalidades[i]) for i in top_modalidades]
        # Extrai a publicação
        prob_publicacoes = np.sum(publicacoes_head_model.predict(vectors) * publicacoes_model.predict(vectors), axis=0)
        top_publicacoes = np.argsort(-prob_publicacoes)[:topn]
        publicacoes = [(lista_tipo_publicacao[i], prob_publicacoes[i]) for i in top_publicacoes]
        # Extrai a probabilidade de festividade
        probabilidade_festividade = np.max(festividade_head_model.predict(vectors))
        return (modalidades, publicacoes, probabilidade_festividade)

In [19]:
publicacao_de_teste = """REFEITURA MUNICIPAL DE SANHARÓ - GABINETE DO
PREFEITO
PORTARIA Nº 136/2023 15 DE SETEMBRO DE 2023
REVOGA PORTARIA Nº 114 DE, 09 DE AGOSTO
DE 2023.
O PREFEITO DO MUNICÍPIO DE SANHARÓ, Estado de
Pernambuco, no uso de suas atribuições legais que a legislação
vigente lhe confere, de acordo com a Lei Orgânica Municipal,
RESOLVE:
Art. 1º - Fica revogada a Portaria nº 114, de 09 de agosto de 2023.
Art. 2º Esta Portaria entra em vigor na data de sua publicação,
retroagindo seus efeitos ao dia 01 de setembro de 2023.
Sanharó/PE, 15 de setembro de 2023.
CÉSAR AUGUSTO DE FREITAS
Prefeito
Publicado por:
Tamires da Silva Soares
Código Identificador:3BB48AEF"""

In [20]:
m,p,f = avaliar(publicacao_de_teste)
print(m)
print(p)
print(f)

[('Dispensa de Licitação', 0.00014369957), ('Aviso de Chamamento Público', 3.8607413e-06), ('Pregão', 1.812853e-06)]
[('Portaria', 2.4023206), ('Decreto', 0.5192995), ('Errata', 0.027407402)]
0.0014594189
